# Imports

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import os
import pdb
import pickle

import tensorflow as tf
from tensorflow.python.client import device_lib
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.models import Sequential, Model, load_model
from keras.layers import Activation, Dense, Input, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from contextlib import redirect_stdout

# Functions for predicting the mean, standard deviation of 2D Gaussian

## predict() -- generate new Gaussians, apply model, plot results

In [3]:
def predict(model):
    # Make 1000 new Gaussians to apply the model to
    predX, predy = make_gaussians(1000, add_noise=True)
        
    # Apply the model to get predicted means and sigmas of the Gaussians
    pamp, pxmu, pxsig, pymu, pysig = model.predict(predX, batch_size=None, verbose=0)
    
    # Check distribution of difference between true and predicted means, sigmas
    plt.subplot(511)
    _, _, _ = plt.hist(predy[:, 0] - pamp.flatten(), bins=30)
    plt.subplot(512)
    _, _, _ = plt.hist(predy[:, 1] - pxmu.flatten(), bins=30)
    plt.subplot(513)
    _, _, _ = plt.hist(predy[:, 2] - pxsig.flatten(), bins=30)
    plt.subplot(514)
    _, _, _ = plt.hist(predy[:, 3] - pymu.flatten(), bins=30)
    plt.subplot(515)
    _, _, _ = plt.hist(predy[:, 4] - pysig.flatten(), bins=30)
    
    plt.show()
    
    # Check the relation between true and predicted means, sigmas
    oto_amp = np.linspace(1., 10., 32) #one-to-one relation for amplitudes
    oto_means = np.linspace(-1., 1., 32) # one-to-one relation for means
    oto_sigmas = np.linspace(0.25, 4.0, 32) # one-to-one relation for sigmas
    
    fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(15, 4))
    
    # Plot the true y's and predicted y's from the NN model
    ax1.scatter(predy[:, 0], pamp.flatten(), marker='.')
    # Plot the 1-to-1 line
    ax1.plot(oto_amp, oto_amp, color='black', ls='--')
    ax1.set_xlabel('True value')
    ax1.set_ylabel('Predicted value')
    ax1.set_title(r'Gaussian Amplitude')

    ax2.scatter(predy[:, 1], pxmu.flatten(), marker='.')
    ax2.plot(oto_means, oto_means, color='black', ls='--')
    ax2.set_xlabel('True value')
    ax2.set_ylabel('Predicted value')
    ax2.set_title(r'Gaussian x-$\mu$')
    
    ax3.scatter(predy[:, 2], pxsig.flatten(), marker='.')
    ax3.plot(oto_sigmas, oto_sigmas, color='black', ls='--')
    ax3.set_xlabel('True value')
    ax3.set_ylabel('Predicted value')
    ax3.set_title(r'Gaussian x-$\sigma$')
    
    ax4.scatter(predy[:, 3], pymu.flatten(), marker='.')
    ax4.plot(oto_means, oto_means, color='black', ls='--')
    ax4.set_xlabel('True value')
    ax4.set_ylabel('Predicted value')
    ax4.set_title(r'Gaussian y-$\mu$')
    
    ax5.scatter(predy[:, 4], pysig.flatten(), marker='.')
    ax5.plot(oto_sigmas, oto_sigmas, color='black', ls='--')
    ax5.set_xlabel('True value')
    ax5.set_ylabel('Predicted value')
    ax5.set_title(r'Gaussian y-$\sigma$')

    fig.show()

## Generate X_train, y_train, X_test, y_test

### Generate 2D Gaussians given mus, sigmas

In [4]:
# Calculate a 2d Gaussian given its mean and standard deviation
def gaussian2d(x_vals, y_vals, amp, x_mu, x_sigma, y_mu, y_sigma):
    return amp * np.exp(-0.5 * ( ((x_vals - x_mu)/x_sigma)**2 + ((y_vals - y_mu)/y_sigma)**2 ))

# Make array that describes Gaussian
def make_gaussians(num, amp_min=1., amp_max = 10.0, mu_min=-1.0, mu_max=1.0, sig_min=0.25, sig_max=4.0, add_noise=False): 

    amp = np.random.uniform(amp_min, amp_max, num)
    
    x_mus = np.random.uniform(mu_min, mu_max, num)
    x_sigmas = np.random.uniform(sig_min, sig_max, num)
    
    y_mus = np.random.uniform(mu_min, mu_max, num)
    y_sigmas = np.random.uniform(sig_min, sig_max, num)

    x_vals = np.linspace(-10.0, 10.0, 32)
    y_vals = np.linspace(-10.0, 10.0, 32)
    
    x_grid, y_grid = np.meshgrid(x_vals, y_vals)
    
    models = np.zeros((num, 32, 32))
    noise = np.zeros((num, 32, 32))
    noise_stds = np.random.uniform(0.0, sig_max, num)
    
    #if add_noise:
        # Add some noise -- currently noise is Gaussian but all with same standard deviation = 1.0
    #    noise = np.random.normal(0.0, 1.0, models.shape)

    for i in range(num):
        models[i] = gaussian2d(x_grid, y_grid, amp[i], x_mus[i], x_sigmas[i], y_mus[i], y_sigmas[i])
        
        if add_noise:
            noise[i] = np.random.normal(0.0, noise_stds[i], models[0].shape)
    
    # Also want to save and return the true means, sigmas used for the Gaussians
    targets = np.vstack((amp, x_mus, x_sigmas, y_mus, y_sigmas)).T
    
    models = models.reshape(num, 32, 32, 1)
    noise = noise.reshape(num, 32, 32, 1)

    return models + noise, targets

### generate_dataset() -- wrapper to make n_train, n_test Gaussians and reshape

In [5]:
# Create training and test sets
def generate_dataset(n_train=10000, n_test=1000):
    X_train, y_train = make_gaussians(n_train, add_noise=True)
    X_test, y_test = make_gaussians(n_test, add_noise=True)
    
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
    
    return X_train, y_train, X_test, y_test

## hyperparams() -- generate random hyperparameters

In [21]:
def hyperparam(mnum):
    '''
    Generate a random set of hyperparameters
    
    Input
    -----
    mnum (int) : model index number
    
    Returns
    -------
    hyperpar : dict
    
    '''
    # Define all of the allowed parameter space
    allowed_hpars = dict(learning_rate      = [0.0005, 0.0007, 0.0010, 0.0030, 0.0050, 0.0070, 0.0100],
                         lr_decay           = [0.0, 1.0],
                         num_epochs         = [30, 50, 100],
                         batch_size         = [16, 32, 64, 128], #100, 500, 1000, 2000, 5000],

                         # Number of filters in each convolutional layer
                         conv_filter_1 = [2, 4, 8, 16], #32, 64], 
                         conv_filter_2 = [1, 2, 4, 8], #16, 32, 64], 
                         
                         # Kernel size
                         conv_kernel_1 = [2, 3, 4, 5],
                         conv_kernel_2 = [1, 2, 3, 4],
                         
                         # Stride of each kernal
                         #conv_stride_1 = [1, 2, 4, 6],
                         #conv_stride_2 = [1, 2, 4, 6],
                         #conv_stride_3 = [1, 2, 4, 6],
                         
                         # Pooling kernel size
                         pool_kernel_1 = [2, 3, 4, 5],

                         # Pooling stride
                         #pool_stride_1 = [1, 2, 3],
                         #pool_stride_2 = [1, 2, 3],
                         #pool_stride_3 = [1, 2, 3],
                         
                         # Fully connected layers
                         fc1_neurons   = [64, 128, 256, 512],
                         )
    
    # Generate dictionary of values
    hyperpar = dict({})
    for key in allowed_hpars.keys():
        hyperpar[key] = np.random.choice(allowed_hpars[key])
    
    print ('Hyperparameters:')
    print (hyperpar)
    
    # Save these parameters and return the hyperpar
    save_obj(hyperpar, 'model_{0:03d}_hyperparams'.format(mnum))
    print ('Saved hyperparameters!')
    
    return hyperpar

## build_model() -- given random hyperparameters, build the model

In [10]:
def build_model(hyperpar):
    # Extract parameters from hyperparameter set
    conv1_filter = hyperpar['conv_filter_1']
    conv2_filter = hyperpar['conv_filter_2']
    
    conv1_kernel = hyperpar['conv_kernel_1']
    conv2_kernel = hyperpar['conv_kernel_2']

    pool1_kernel = hyperpar['pool_kernel_1']
    
    fc1_neurons = hyperpar['fc1_neurons']

    # Build model
    print ('Building model from hyperparameters...')
    input1 = Input(shape=(32, 32, 1))#X_train.shape[1], X_train.shape[2], X_train.shape[3])) # this returns the 32x32 2d array of Gaussian
    
    conv1 = Conv2D(filters=conv1_filter, kernel_size=(conv1_kernel, conv1_kernel), activation='relu')(input1)
    conv2 = Conv2D(filters=conv2_filter, kernel_size=(conv2_kernel, conv2_kernel), activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(pool1_kernel, pool1_kernel))(conv2)    
    
    flat1 = Flatten()(pool1)
    fc1 = Dense(fc1_neurons, activation='relu')(flat1)
    
    out1 = Dense(1, activation='linear', name='amplitude')(fc1)
    out2 = Dense(1, activation='linear', name='x_mean')(fc1)
    out3 = Dense(1, activation='linear', name='x_sigma')(fc1)
    out4 = Dense(1, activation='linear', name='y_mean')(fc1)
    out5 = Dense(1, activation='linear', name='y_sigma')(fc1)
    
    model = Model(inputs=input1, outputs=[out1, out2, out3, out4, out5])
    model.summary()

    return model

### Create, compile, fit, and evaluate NN model

## evalute_model() -- read in randomized hyperparameters to compile and fit model

For all struggles with h5py saving models: https://stackoverflow.com/questions/60917467/alternating-errors-using-hdf5-library-and-h5py-module

In [12]:
# fit and evaluate a model
def evaluate_model(X_train, y_train, X_test, y_test, hyperpar, mnum, verbose=1):
    filepath = os.getcwd() + '/' #os.path.dirname(os.path.abspath(__file__))
    model_name = 'model_{0:03d}'.format(mnum)

    # Construct neural network, depending on GPUs
    ngpus = len(get_available_gpus())
    if ngpus > 1:
        model = build_model(hyperpar)
        # Make this work on multiple GPUs
        gpumodel = multi_gpu_model(model, gpus=ngpus)
    else:
        gpumodel = build_model(hyperpar)

    # Summarize layers
    summary = False
    if summary:
        with open(filepath + model_name + '.summary', 'w') as f:
            with redirect_stdout(f):
                model.summary()
    # Plot graph
    plotit = False
    if plotit:
        pngname = filepath + model_name + '.png'
        plot_model(model, to_file=pngname)

    # Compile model
    decay = hyperpar['lr_decay']*hyperpar['learning_rate'] / hyperpar['num_epochs']
    optadam = Adam(lr=hyperpar['learning_rate'], decay=decay)

    gpumodel.compile(loss='mse', optimizer=optadam)#, metrics=['mean_squared_error'])

    # Initialise callbacks
    ckp_name = filepath + model_name + '.hdf5'
    sav_name = filepath + model_name #+ '_save.hdf5'
    csv_name = filepath + model_name + '.log'
    checkpointer = ModelCheckpoint(filepath=ckp_name, verbose=1, save_best_only=True)
    csv_logger = CSVLogger(csv_name, append=True)

    # Fit network
    gpumodel.fit(X_train, [y_train[:,0], y_train[:,1], y_train[:,2], y_train[:,3], y_train[:,4]],
        epochs=10, verbose=verbose)#hyperpar['num_epochs'], verbose=verbose)

    gpumodel.save(sav_name)
    print ('Saved model ' + sav_name)
    
    # Evaluate model
    accuracy = gpumodel.evaluate(X_test, [y_test[:,0], y_test[:,1], y_test[:,2], y_test[:,3], y_test[:,4]], 
        batch_size=hyperpar['batch_size'], verbose=0)

    return accuracy, gpumodel.metrics_names

In [13]:
def restart_model(model_name, X_train, y_train, X_test, y_test, epochs=50, batch_size=16, verbose=1):
    # Load model    
    filepath = os.getcwd() + '/' #os.path.dirname(os.path.abspath(__file__))+'/'
    loadname = filepath + model_name + '.hdf5'
    model = load_model(loadname, compile=False)

    # Make this work on multiple GPUs
    #gpumodel = multi_gpu_model(model, gpus=4)

    gpumodel.compile(loss='mse', optimizer='adam')#, metrics=['mean_squared_error'])
    
    # Initialise callbacks
    ckp_name = filepath + model_name + '_chkp_restart.hdf5'
    sav_name = filepath + model_name + '_save_restart.hdf5'
    csv_name = filepath + model_name + '_restart.log'
    checkpointer = ModelCheckpoint(filepath=ckp_name, verbose=1, save_best_only=True)
    csv_logger = CSVLogger(csv_name, append=True)
    
    # Fit network
    gpumodel.fit(X_train, [y_train[:,0], y_train[:,1], y_train[:,2], y_train[:,3], y_train[:,4]],
        epochs=epochs, verbose=verbose)

    gpumodel.save(sav_name)

    # Evaluate model
    accuracy = gpumodel.evaluate_generator(X_test, [y_test[:,0], y_test[:,1], y_test[:,2], y_test[:,3], y_test[:,4]], 
        batch_size=batch_size, verbose=0)
    
    return accuracy, gpumodel.metrics_names

In [18]:
# If relevant, check accuracy of the model
# summarize scores
def summarize_results(scores):
    keys = scores.keys()
    
    for ii in keys:
        m, s = np.mean(scores[ii]), np.std(scores[ii])
        
        print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [15]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

# Save and load model
def save_obj(obj, dirname):
    with open(dirname + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(dirname):
    with open(dirname + '.pkl', 'rb') as f:
        return pickle.load(f)

In [19]:
# Detect features in a dataset
def localise_features(mnum, repeats=3, restart=False):
    # Generate hyperparameters
    hyperpar = hyperparam(mnum)
    print ('Generated hyperparameters \n')
    
    # Generate data
    X_train, y_train, X_test, y_test = generate_dataset()
    print ('Generated dataset \n')
    
    # repeat experiment
    allscores = dict({})
    
    for r in range(repeats):
        if restart:
            model_name = 'gaussian2d_save'
            scores, names = restart_model(model_name, X_train, y_train, X_test, y_test)
        else:
            scores, names = evaluate_model(X_train, y_train, X_test, y_test, hyperpar, mnum)
        if r == 0:
            for name in names:
                allscores[name] = []
        for ii, name in enumerate(names):
            allscores[name].append(scores[ii] * 100.0)
            if '_acc' in name:
                print('%s >#%d: %.3f' % (name, r + 1, allscores[name][-1]))
            else:
                print('%s >#%d: %.3f' % (name, r + 1, scores[ii]))
                
    # Summarize results
    summarize_results(allscores)

In [20]:
# Set the number of epochs
if False:
    # Generate data
    generate_dataset()
else:
    # Once the data exist, run the experiment
    m_init = 0
    # mnum sets the number of model fitting + evaluations using *different* sets of hyperparameters
    mnum = m_init
    while True:
        try:
            print (mnum)
            # repeats sets the number of model fitting + evaluations using the same set of hyperparameters
            # but with different initial starting points/values for the trainable parameters (weights, biases, etc.)
            localise_features(mnum, repeats=1, restart=False)
        except ValueError:
            continue
        mnum += 1
        if mnum >= m_init+10:
            break

0
Hyperparameters:
{'learning_rate': 0.005, 'lr_decay': 0.0, 'num_epochs': 30, 'batch_size': 128, 'conv_filter_1': 16, 'conv_filter_2': 32, 'conv_kernel_1': 3, 'conv_kernel_2': 5, 'pool_kernel_1': 3, 'fc1_neurons': 64}
Saved hyperparameters!
Generated hyperparameters 

Generated dataset 

Building model from hyperparameters...
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 30, 30, 16)   160         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 26, 26, 32)   12832  

Epoch 1/10
10000/10000 [==============================] - 20s 2ms/step - loss: 3.5337 - amplitude_loss: 1.7601 - x_mean_loss: 0.2230 - x_sigma_loss: 0.6856 - y_mean_loss: 0.2267 - y_sigma_loss: 0.6352
Epoch 2/10
10000/10000 [==============================] - 19s 2ms/step - loss: 1.6882 - amplitude_loss: 0.8799 - x_mean_loss: 0.1305 - x_sigma_loss: 0.2768 - y_mean_loss: 0.1265 - y_sigma_loss: 0.2741
Epoch 3/10
10000/10000 [==============================] - 17s 2ms/step - loss: 1.3644 - amplitude_loss: 0.7060 - x_mean_loss: 0.1160 - x_sigma_loss: 0.2169 - y_mean_loss: 0.1118 - y_sigma_loss: 0.2134
Epoch 4/10
10000/10000 [==============================] - 17s 2ms/step - loss: 1.0943 - amplitude_loss: 0.5358 - x_mean_loss: 0.1042 - x_sigma_loss: 0.1798 - y_mean_loss: 0.0973 - y_sigma_loss: 0.1774
Epoch 5/10
10000/10000 [==============================] - 16s 2ms/step - loss: 0.9012 - amplitude_loss: 0.4175 - x_mean_loss: 0.0926 - x_sigma_loss: 0.1553 - y_mean_loss: 0.0894 - y_sigma_loss: 0.

Epoch 1/10
10000/10000 [==============================] - 11s 1ms/step - loss: 3.8887 - amplitude_loss: 2.2224 - x_mean_loss: 0.2235 - x_sigma_loss: 0.6012 - y_mean_loss: 0.2484 - y_sigma_loss: 0.5891
Epoch 2/10
10000/10000 [==============================] - 10s 1ms/step - loss: 1.8973 - amplitude_loss: 1.0348 - x_mean_loss: 0.1344 - x_sigma_loss: 0.2940 - y_mean_loss: 0.1424 - y_sigma_loss: 0.2904 4s
Epoch 3/10
10000/10000 [==============================] - 10s 976us/step - loss: 1.6427 - amplitude_loss: 0.8977 - x_mean_loss: 0.1247 - x_sigma_loss: 0.2500 - y_mean_loss: 0.1235 - y_sigma_loss: 0.2469
Epoch 4/10
10000/10000 [==============================] - 10s 1ms/step - loss: 1.5152 - amplitude_loss: 0.8007 - x_mean_loss: 0.1254 - x_sigma_loss: 0.2294 - y_mean_loss: 0.1211 - y_sigma_loss: 0.2380
Epoch 5/10
10000/10000 [==============================] - 10s 1ms/step - loss: 1.4299 - amplitude_loss: 0.7530 - x_mean_loss: 0.1136 - x_sigma_loss: 0.2220 - y_mean_loss: 0.1145 - y_sigma_los

Generated dataset 

Building model from hyperparameters...
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 30, 30, 64)   640         input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 27, 27, 8)    8200        conv2d_13[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_7 (MaxPooling2D)  (None, 9, 9, 8)      0           conv2d_14[0][0]                  
_________________________________

10000/10000 [==============================] - 11s 1ms/step - loss: 9.4384 - amplitude_loss: 5.2250 - x_mean_loss: 1.1053 - x_sigma_loss: 0.8279 - y_mean_loss: 0.3230 - y_sigma_loss: 1.9485
Epoch 2/10
10000/10000 [==============================] - 11s 1ms/step - loss: 2.2821 - amplitude_loss: 1.2555 - x_mean_loss: 0.1527 - x_sigma_loss: 0.3597 - y_mean_loss: 0.1458 - y_sigma_loss: 0.3706
Epoch 3/10
10000/10000 [==============================] - 10s 1ms/step - loss: 2.0550 - amplitude_loss: 1.1328 - x_mean_loss: 0.1412 - x_sigma_loss: 0.3189 - y_mean_loss: 0.1409 - y_sigma_loss: 0.3202
Epoch 4/10
10000/10000 [==============================] - 10s 1ms/step - loss: 1.8953 - amplitude_loss: 1.0255 - x_mean_loss: 0.1392 - x_sigma_loss: 0.2989 - y_mean_loss: 0.1352 - y_sigma_loss: 0.2961
Epoch 5/10
10000/10000 [==============================] - 11s 1ms/step - loss: 1.8250 - amplitude_loss: 0.9871 - x_mean_loss: 0.1348 - x_sigma_loss: 0.2928 - y_mean_loss: 0.1303 - y_sigma_loss: 0.2808
Epoch 

Epoch 1/10
10000/10000 [==============================] - 16s 2ms/step - loss: 5.1667 - amplitude_loss: 3.0076 - x_mean_loss: 0.2548 - x_sigma_loss: 0.7873 - y_mean_loss: 0.2421 - y_sigma_loss: 0.8689
Epoch 2/10
10000/10000 [==============================] - 15s 1ms/step - loss: 2.2630 - amplitude_loss: 1.2105 - x_mean_loss: 0.1525 - x_sigma_loss: 0.3772 - y_mean_loss: 0.1657 - y_sigma_loss: 0.3565
Epoch 3/10
10000/10000 [==============================] - 15s 2ms/step - loss: 1.8907 - amplitude_loss: 1.0388 - x_mean_loss: 0.1357 - x_sigma_loss: 0.2945 - y_mean_loss: 0.1410 - y_sigma_loss: 0.2808
Epoch 4/10
10000/10000 [==============================] - 16s 2ms/step - loss: 1.7186 - amplitude_loss: 0.9374 - x_mean_loss: 0.1278 - x_sigma_loss: 0.2592 - y_mean_loss: 0.1321 - y_sigma_loss: 0.2609
Epoch 5/10
10000/10000 [==============================] - 15s 2ms/step - loss: 1.5888 - amplitude_loss: 0.8656 - x_mean_loss: 0.1202 - x_sigma_loss: 0.2450 - y_mean_loss: 0.1232 - y_sigma_loss: 0.

# Plot 2D Gaussian

In [ ]:
# Import for 3d plotting
from mpl_toolkits import mplot3d

In [ ]:
# Create x, y grid, and use it to generate a 2d Gaussian
x_vals = np.linspace(-10, 10, 32)
y_vals = np.linspace(-10, 10, 32)

x, y = np.meshgrid(x_vals, y_vals)

z = gaussian2d(x, y, 1, 0, 2, 0, 2)

# Plot 2d Gaussian
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection='3d')

ax.plot_surface(x, y, z, rstride=1, cstride=2, cmap='winter')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()